In [318]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import random

my_header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

In [339]:
%%time
def sleep():
    time.sleep(random.randint(1,2))
    
group_list = [('Medical Exam Gloves', 'https://products.halyardhealth.com/infection-prevention/medical-exam-gloves.html'),
              ('Patient Care', 'https://products.halyardhealth.com/infection-prevention/patient-care.html'),
              ('Facial & Respiratory Protection', 'https://products.halyardhealth.com/infection-prevention/facial-respiratory-protection.html'),
              ('Protective Apparel', 'https://products.halyardhealth.com/infection-prevention/protective-apparel.html')]
mfg = 'Halyard Health'

#      (group)        >   (subgroup)   >    (product line)     > (product)
# medical exam gloves > nitrile gloves > purple nitrile gloves > 55081

df_total = pd.DataFrame()

for group in group_list:
    group_title = group[0]
    group_url = group[1]  
    
    sleep()
    print('Scraping ', group_title)
    
    group_response = requests.get(group_url, headers=my_header)
    group_text = BeautifulSoup(group_response.text, 'html.parser')
    sub_group_list = group_text.find('ul', {'class':'category clear'}).find_all('li')
    sub_group_list = [(t.h3.get_text(), t.a['href']) for t in sub_group_list]
    
    for sub_group in sub_group_list:
        sub_group_title = sub_group[0]
        sub_group_url = sub_group[1]     
        
        sleep()
        print('\tScraping ', sub_group_title)

        sub_group_response = requests.get(sub_group_url, headers=my_header)
        sub_group_text = BeautifulSoup(sub_group_response.text, 'html.parser')
        product_lines = sub_group_text.find('div', {'class': 'category-products'}).find_all('li', {'class': 'mix'})
        product_lines = [(t.img['alt'], t.a['href']) for t in product_lines]

        for product in product_lines:
            product_title = product[0]
            product_url = product[1]
            
            sleep()
            print('\t\tScraping ', product_title)

            product_response = requests.get(product_url, headers=my_header)
            product_text = BeautifulSoup(product_response.text, 'html.parser')
            if product_text.get_text().find('404 Page Not Found') >= 0:
                print('\t\t\t404 Page Not Found')
                continue
            if product_text.get_text().find('Narrow Your Selections') >= 0:
                num_options = int(product_text.find('div', {'id':'total-item-count'}).get_text()[0:1])
            else:
                num_options = 0

            if num_options <= 1:
                product_list = product_text.find('span', {'class':'product-sku'}).get_text()
                desc_table = product_text.find('div', {'class':'product-attribute-description'})
                if desc_table:
                    desc_table = desc_table.find_all('div')
                else:
                    desc_table = []
                spec_table = product_text.find('table', {'id':'product-attribute-specs-table'}).find_all('tr')

                desc1 = ', '.join([t.get_text().strip() for t in desc_table])
                desc2 = ', '.join([t.get_text().strip().replace('\n', ':') for t in spec_table])
                description = desc1 + ', ' + desc2
                df_product = pd.DataFrame({'mfg':[mfg],
                                           'group':[group_title],
                                           'sub-group':[sub_group_title],
                                           'product_line': [product_title],
                                           'product_number':[product_list],
                                           'description':[description],
                                           'url':[product_url]})
            else:
                product_table = product_text.find('ul',{'id':'mix-grid'})
                product_list = [t.string for t in product_table.find_all('span', {'class':'product-sku'})]
                description_table = [t for t in product_table.find_all('div',{'class':'product-attribute-description'})]
                spec_table = product_text.find('table', {'id':'product-attribute-specs-table'}).find_all('tr')
                description_list = []
                desc2 = ', '.join([t.get_text().strip().replace('\n', ':') for t in spec_table])
                for descr in description_table:
                    descr = descr.find_all('div', {'class':'attribute-group'})
                    desc1 = ', '.join([t.get_text().strip().replace('\n', '') for t in descr])
                    description_list.append(desc1 + ', ' + desc2)
                df_product = pd.DataFrame({'mfg':mfg,
                                           'group':group_title,
                                           'sub-group':sub_group_title,
                                           'product_line': product_title,
                                           'product_number':product_list,
                                           'description':description_list,
                                           'url': product_url})
            print('\t\t\t', df_product.iloc[0].product_number)
            print('\t\t\t', df_product.iloc[0].description)
            print('\t\t\t', df_product.iloc[0].url)
            df_total = df_total.append(df_product)
print('Done!')
df_total = df_total.reset_index(drop=True)
df_total

Scraping  Medical Exam Gloves
	Scraping  Nitrile
		Scraping  PURPLE NITRILE* Exam Glove 
			404 Page Not Found
		Scraping  STERLING ZERO* Nitrile Exam Glove
			 47459
			 Size:X-Small, Brand:STERLING ZERO*, Color:Sterling, Name:STERLING ZERO* Nitrile Exam Glove, Dispenser:SMARTPULL*, Cleared for Use with Chemotherapy Drugs:Yes, Expected Exposure to Fluids or Chemicals:Low to High, Acceptable Quality Level (AQL) :1.0 AQL, ASTM D5151 Test Standards:Meets or Exceeds, ASTM D6319 Test Standards:Meets or Exceeds, Biocompatibility-ISO Standard for Primary Skin Irritation & Sensitivity :Meets or Exceeds, NFPA 1999-2013 Certified:No, Material:Nitrile, Powder Free:Yes, Natural Rubber Latex-Free:Yes, Cuffed and Walleted:No, Cuff:Beaded Cuff, Glove Length:9.5 in., Gauge Thickness Middle Fingertip:.10 mm 3.9 mil, Gauge Thickness Palm :.07mm 2.8 mil, Ultimate Elongation:Before Aging:  550%; After Aging:  500%, Tensile Strength:Before Aging:  24 MPa; After Aging:  21 MPa, Texture Status:Textured Fing

		Scraping  HALYARD* BLUE NITRILE* Exam Glove
			 53100
			 Size:X-Small, Brand:HALYARD*, Brand:HALYARD*, UNSPSC Classification:42132203, Color:Blue, Name:BLUE NITRILE* Exam Glove, Sterile Options Available:No, Expected Exposure to Fluids or Chemicals:Moderate to High, Acceptable Quality Level (AQL) :Meets or Exceeds, ASTM D6319 Test Standards:Meets or Exceeds, Biocompatibility-ISO Standard for Primary Skin Irritation & Sensitivity :Meets or Exceeds, Material:Nitrile, Powder Free:Yes, Natural Rubber Latex-Free:Yes, Cuffed and Walleted:No, Cuff:Beaded Cuff, Glove Length:9.5 in., Gauge Thickness Middle Fingertip:.15mm 5.9 mil, Gauge Thickness Palm :.12mm 4.7 mil, Ultimate Elongation:Before Aging:  550%; After Aging:  500%, Tensile Strength:Before Aging:  21 MPa; After Aging:  21 MPa, Texture Status:Textured Fingertips, Weight:Heavy-Weight, Fit Type:Ambidextrous, Boxes Per Case:10
			 https://products.halyardhealth.com/infection-prevention/medical-exam-gloves/nitrile/blue-nitrile-exam-glo

		Scraping  JUMBO-PLUS* Replacement Bladder
			 33725
			 , UNSPSC Classification:42142101, Natural Rubber Latex-Free:Yes, Sterilization Method:N/A, UK SKU:P3372507, Name:JUMBO-PLUS* Replacement Bladder, Brand:JUMBO-PLUS*, Use Frequency :Single Use, Length (in.):8.5, Width (in.):11, Packaging:Carton, Units per Case:24
			 https://products.halyardhealth.com/infection-prevention/patient-care/cold-therapy/jumbo-plus-replacement-bladder.html
		Scraping  JUMBO-PLUS* Ice Pack, Dual Chamber
			 33724
			 , UNSPSC Classification:42142101, Natural Rubber Latex-Free:Yes, Sterilization Method:N/A, UK SKU:P3372407, Name:JUMBO-PLUS* Ice Pack, Dual Chamber, Brand:JUMBO-PLUS*, Use Frequency :Single Use, Size:Dual Chamber, Length (in.):11, Width (in.):21, Closure:Hook and Loop, Area of Use:Lower Back; Abdomen and Chest; Knee, Elbows & Ankles, Strap Quantity:2, Packaging:Poly Bag, Units per Case:10
			 https://products.halyardhealth.com/infection-prevention/patient-care/cold-therapy/jumbo-plus-ice-pack

	Scraping  Telemetry Unit Pouches
		Scraping  Telemetry Unit Pouch
			 71825
			 , UNSPSC Classification:42181719, Natural Rubber Latex-Free:Yes, Name:Telemetry Unit Pouch, Sterilization Method:N/A, Use Frequency:Single Use, Size:Universal, Closure:Ties, Packaging:Pouch
			 https://products.halyardhealth.com/infection-prevention/patient-care/telemetry-unit-pouches/telemetry-unit-pouch-standard.html
		Scraping  CLEARVIEW* Telemetry Unit Pouch
			 71900
			 , UNSPSC Classification:42181719, Natural Rubber Latex-Free:Yes, Name:CLEARVIEW* Telemetry Unit Pouch, Sterilization Method:N/A, Use Frequency:Single Use, Size:Fits Any Transmitter, Closure:Ties, Front Panel:Clear, Packaging:Poly Bag
			 https://products.halyardhealth.com/infection-prevention/patient-care/telemetry-unit-pouches/telemetry-unit-pouch-with-ties-clearview.html
		Scraping  Telemetry Unit Pouch, Hook and Loop
			 71800
			 , UNSPSC Classification:42181719, Natural Rubber Latex-Free:Yes, UK SKU:P7180007, Name:Telemetry Unit 

		Scraping  FLUIDSHIELD* Level 2 Expanded Chamber Surgical Mask with SO SOFT* Lining
			 39123
			 , Brand:FLUIDSHIELD*, Sterile:No, Color:Blue and White, Level of Protection:Level 2, Name:FLUIDSHIELD* Level 2 Expanded Chamber Surgical Mask with SO SOFT* Lining, Type:Surgical, FLUIDSHIELD* :Yes, Style:Pouch, SO SOFT* Lining:Yes, SAFETY SEAL*:No, Size:Regular, Fluid Resistant:Yes, Fluid Protection:Moderate, ASTM F2100-11:Yes, ASTM Level of Protection:2, BFE (3):99.4%, SPLASH RESISTANCE (80,120,160):120, DELTA P:Pass, PFE (.1):98.7%, Items Per Box:50, Boxes Per Case:6, Items Per Case:300
			 https://products.halyardhealth.com/infection-prevention/facial-respiratory-protection/masks-respirators/fluidshield-level-2-expanded-chamber-surgical-mask-with-so-soft-lining.html
		Scraping  FLUIDSHIELD* Level 3 Fog-Free Procedure Mask with SO SOFT* Lining, Visor, Orange
			 47137
			 , Brand:FLUIDSHIELD* Level 3, UNSPSC Classification:42131606, Sterile:No, Color:Orange, Name:FLUIDSHIELD* Level 3 Fo

...

	Scraping  Qualitative Fit Test & Accessories
		Scraping  FLUIDSHIELD* Ported N95 Particulate Filter Respirator
			 99181
			 SAFETY SEAL*:No, Size:Small, Brand:FLUIDSHIELD*, UNSPSC Classification:42131606, Sterile:No, Color:Orange, Type:Respirator, Level of Protection:Level 3, Name:FLUIDSHIELD* Ported N95 Particulate Filter Respirator, Type:Respirator, FLUIDSHIELD* :Yes, RHP Style:Pouch, SO SOFT* Lining:Yes, Ported:Yes, Fog Resistance:N/A, Fluid Resistant:Yes, RHP Fluid Protection:High, ASTM Level of Protection:3, BFE (3):≥ 99%, SPLASH RESISTANCE (80,120,160):160, PFE (.1):≥ 99%, Items Per Box:100, Boxes Per Case:1, Items Per Case:100
			 https://products.halyardhealth.com/infection-prevention/facial-respiratory-protection/qualitative-fit-test-accessories/fluidshield-ported-n95-particulate-filter-respirator.html
		Scraping  Qualitative Fit Test Kit
			 47950
			 , Brand:HALYARD*, UNSPSC Classification:42131606, Sterile:No, Name:Qualitative Fit Test Kit, UK SKU:P4795000, Fluid Resistan

		Scraping  HALYARD* Medium Weight Tri-Layer Over-the-Head Isolation Gown
			 32500
			 Size:Universal, Chest:28.5 in. / 72.5 cm., Sleeve Length:24 in. / 60.75 cm., Length:46 in. / 116 cm., Brand:HALYARD*, Name:HALYARD* Medium Weight Tri-Layer Over-the-Head Isolation Gown, Coverage:Open Back, Rating:Not Rated, Fabric:3-Layer SMS, Color:Yellow, Seam Closure:Heat Seal, Tie Waist:Yes, Cuff End:Thumb Hooks and Elastic Cuffs
			 https://products.halyardhealth.com/infection-prevention/protective-apparel/protective-gowns/halyard-medium-weight-tri-layer-over-the-head-isolation-gown.html
		Scraping  HALYARD* Light Weight Tri-Layer Over-the-Head Isolation Gown
			 32502
			 Size:Universal, Chest:28.5 in. / 72.5 cm., Length:46 in. / 116 cm., Sleeve Length:24 in. / 60.75 cm., Brand:HALYARD*, Name:HALYARD* Light Weight Tri-Layer Over-the-Head Isolation Gown, Coverage:Open Back, Rating:Not Rated, Fabric:3-Layer SMS, Color:Yellow, Seam Closure:Heat Seal, Neck Closure:Over-the-Head, Tie Waist:Yes, Cuf

		Scraping  HALYARD BASICS* Tri-Layer AAMI2 Isolation Gown
			 13961
			 Size:X-Large, Dimensions (in., Chest Width / Length / Sleeve:30 / 47 / 24, Brand:HALYARD BASICS*, UNSPSC Classification:42131612, Name:HALYARD BASICS* Tri-Layer AAMI2 Isolation Gown, Coverage:Full Coverage, Rating:AAMI Level 2, Fabric:3-Layer SMS, Color:Yellow, Back Gather:Yes, Recommended Areas of Use:ICU, Hyperbaric and Dialysis Unit, Contact Isolation, Med/Surgery Unit, Nursery, Lab and Pathology, Recommended Tasks:Blood Draw, Suturing, Inserting I.V. Lines, Specimen Handling, Arterial Blood Draw, Made in USA Label:Yes, Seam Closure:Hot Melt Glued, Neck Closure:Tape-Tab, Cuff End:Elastic, Items Per Package:10, Packages Per Case:10 Poly Bags, Units per Case:100
			 https://products.halyardhealth.com/infection-prevention/protective-apparel/protective-gowns/halyard-basics-tri-layer-aami2-isolation-gown.html
		Scraping  HALYARD* Tri-Layer Over-the-Head Isolation Gowns
			 29516
			 Size:XX-Large, Color:Yellow, Dime

		Scraping  HALYARD* BASIC* Plus Lab Coat with Knit Collar and Cuffs
			 10020
			 Size:Small, Color:White, Chest:54.2 cm., Front Length:104.1 cm., Sleeve Length:44.5 cm, Brand:HALYARD*, Brand:HALYARD*, UNSPSC Classification:42131607, Sterile:No, Name:BASIC* Plus Lab Coat, AAMI Level:None, Expected Exposure to Fluids or Chemicals:Low, Fabric:3-Layer SMS, Snaps:5, Pockets:3 reinforced, Cuff End:Knit, Collar:Knit, Size/Name Label:Yes, Neck Loop for Hanging:Yes, Seam Closure:Sewn, Sleeve Type:Raglan, Items Per Package:25, Packages Per Case:1 Poly Bag, Units per Case:25
			 https://products.halyardhealth.com/infection-prevention/protective-apparel/lab-coats-lab-jackets/basic-plus-lab-coat-with-knit-collar-and-cuffs.html
		Scraping  Universal Precautions Lab Coat
			 10040
			 Size:Small, Color:White, Chest:54.2 cm., Front Length:105.4 cm., Sleeve Length:47.0 cm, UNSPSC Classification:42131607, Sterile:No, Name:Universal Precautions Lab Coat, AAMI Level:None, Expected Exposure to Fluids or 

,mfg,group,sub-group,product_line,product_number,description,url
0,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47459,"Size:X-Small, Brand:STERLING ZERO*, Color:Ster...",https://products.halyardhealth.com/infection-p...
1,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47460,"Size:Small, Brand:STERLING ZERO*, Color:Sterli...",https://products.halyardhealth.com/infection-p...
2,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47461,"Size:Medium, Brand:STERLING ZERO*, Color:Sterl...",https://products.halyardhealth.com/infection-p...
3,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47462,"Size:Large, Brand:STERLING ZERO*, Color:Sterli...",https://products.halyardhealth.com/infection-p...
4,Halyard Health,Medical Exam Gloves,Nitrile,STERLING ZERO* Nitrile Exam Glove,47463,"Size:X-Large, Brand:STERLING ZERO*, Color:Ster...",https://products.halyardhealth.com/infection-p...
...,...,...,...,...,...,...,...
335,Halyard Health,Protective Apparel,Scrub Pants & Scrub Shirts,Scrub Pants,69714,"Size:XX-Large, Front Length:124.4 cm., UK SKU:...",https://products.halyardhealth.com/infection-p...
336,Halyard Health,Protective Apparel,Scrub Pants & Scrub Shirts,Scrub Shirt,69701,"Size:Medium, Front Length:71.1 cm., UK SKU:P69...",https://products.halyardhealth.com/infection-p...
337,Halyard Health,Protective Apparel,Scrub Pants & Scrub Shirts,Scrub Shirt,69702,"Size:Large, Front Length:71.1 cm., UK SKU:P697...",https://products.halyardhealth.com/infection-p...
338,Halyard Health,Protective Apparel,Scrub Pants & Scrub Shirts,Scrub Shirt,69703,"Size:X-Large, Front Length:77.5 cm., UK SKU:P6...",https://products.halyardhealth.com/infection-p...


In [340]:
# Output to file.
df_total.to_csv('output\df_total.csv', index=False)

In [ ]:
# debugging

# url = 'https://products.halyardhealth.com/infection-prevention/facial-respiratory-protection.html'
# response = requests.get(url, headers=my_header)
# text = BeautifulSoup(response.text, 'html.parser')

# category_list = [(t.img['alt'], t.a['href']) for t in text.find_all('div', {'class': 'cat-image'})]
# for category in category_list:
#     print(category)

# text = text.find('ul', {'class':'category clear'}).find_all('li')
# [(t.h3.get_text(), t.a['href']) for t in text]

# int(text.find('div', {'id':'total-item-count'}).get_text()[0:1])

In [323]:
gtin = pd.read_csv('research/HYH_GTIN_PUBLISHED.csv')
gtin = gtin.dropna(axis=0, how='all')
gtin.loc[:, ['product', 'gtin']] = gtin[['product', 'gtin']].astype(np.int64)
gtin = gtin.astype(str)
gtin = gtin.rename(columns={'product':'product_number'})

In [325]:
# none of the scraped producs are in the gtin file.
np.sum(df_total.merge(gtin, how='left', on='product_number').isna())

group               0
sub-group           0
product_line        0
product_number      0
description         0
url                 0
catalog           340
gtin              340
dtype: int64

# manual fill-in

In [343]:
product_url = 'https://products.halyardhealth.com/infection-prevention/medical-exam-gloves/nitrile/sterling-nitrile-exam-glove-5081.html'
product_response = requests.get(product_url, headers=my_header)
product_text = BeautifulSoup(product_response.text, 'html.parser')
spec_table = product_text.find('table', {'id':'product-attribute-specs-table'}).find_all('tr')

desc = ', '.join([t.get_text().strip().replace('\n', ':') for t in spec_table])
print(desc)

Brand:STERLING*, Name:STERLING* Nitrile Exam Glove, Level:H300, Sterile Options Available:Yes, Expected Exposure to Fluids or Chemicals:Low to Moderate, Acceptable Quality Level (AQL) :Meets or Exceeds, ASTM D5151 Test Standards:N/A, ASTM D6319 Test Standards:Meets or Exceeds, Biocompatibility-ISO Standard for Primary Skin Irritation & Sensitivity :Meets or Exceeds, Material:Nitrile, Powder Free:Yes, Natural Rubber Latex-Free:Yes, Gauge Thickness Palm :.08mm 2.8mil, Ultimate Elongation:Before Aging:  650%; After Aging:  550%, Tensile Strength:Before Aging:  42 MPa; After Aging:  38 MPa, Texture Status:Textured Fingertips, Weight:Medium-Weight
